## Rod Cantilever Example

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["MKL_THREADING_LAYER"] = "GNU"

In [3]:
import numpy as np

import dismech

b = 0.02
h = 0.001

geom = dismech.GeomParams(rod_r0=0.001,
                          shell_h=0,
                          axs=b*h,
                          ixs1=b*h**3/12,
                          ixs2=h*b**3/12,
                          jxs=b*h**3/6)

material = dismech.Material(density=1200,
                            youngs_rod=2e6,
                            youngs_shell=0,
                            poisson_rod=0.5,
                            poisson_shell=0)

static_2d_sim = dismech.SimParams(static_sim=False,
                                  two_d_sim=True,   # can change
                                  use_mid_edge=False,
                                  use_line_search=False,
                                  show_floor=False,
                                  log_data=True,
                                  log_step=1,
                                  dt=1e-2,
                                  max_iter=25,
                                  total_time=1.0,
                                  plot_step=1,
                                  tol=1e-4,
                                  ftol=1e-4,
                                  dtol=1e-2)

env = dismech.Environment()
env.add_force('gravity', g=np.array([0.0, 0.0, -9.81]))

geo = dismech.Geometry.from_txt(
    '../tests/resources/rod_cantilever/horizontal_rod_n51.txt')

robot = dismech.SoftRobot(geom, material, geo, static_2d_sim, env)

### Time Stepping

As we are performing a static simulation, we must change gravity for each time step.

In [4]:
fixed_points = np.array([0, 1, 2, 3, 4, 5])

robot = robot.fix_nodes(fixed_points)
stepper = dismech.ImplicitEulerTimeStepper(robot)
robots = stepper.simulate()

qs = np.stack([robot.state.q for robot in robots])

gradient of strain passed FDM check: False
hessian of strain passed FDM check: False
gradient of strain passed FDM check: False
hessian of strain passed FDM check: False
gradient of strain passed FDM check: False
hessian of strain passed FDM check: False
gradient of strain passed FDM check: False
hessian of strain passed FDM check: False
gradient of strain passed FDM check: False
hessian of strain passed FDM check: False
gradient of strain passed FDM check: False
hessian of strain passed FDM check: False
gradient of strain passed FDM check: False
hessian of strain passed FDM check: False
gradient of strain passed FDM check: False
hessian of strain passed FDM check: False
gradient of strain passed FDM check: False
hessian of strain passed FDM check: False
gradient of strain passed FDM check: False
hessian of strain passed FDM check: False
gradient of strain passed FDM check: False
hessian of strain passed FDM check: False
gradient of strain passed FDM check: False
hessian of strain pass

In [5]:
t = np.arange(robot.sim_params.total_time, step=robot.sim_params.dt)  # (100)
options = dismech.AnimationOptions(y_lim=[-0.01, 0.01],
                  title='Rod Cantilever (N=51)')
fig = dismech.get_interactive_animation_plotly(robot, t, qs, options)
fig.show()

In [6]:
#%matplotlib notebook
#from IPython.display import HTML
#t = np.arange(robot.sim_params.total_time, step=robot.sim_params.dt)  # (100)
#options.title = 'Newmark Beta Rod Cantilever (N=51) (β=0.0.3025, γ=0.6)'
#ani = dismech.get_animation(robot, t[:500], qs[:500], options)
#HTML(ani.to_jshtml())
#ani.save('newmark_beta_rod_cantilever_5s.gif', 'pillow')